In [87]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import glob


In [88]:
# Helper function to read Excel files consistently
def read_excel_files(file_path):
    """Standardized approach to read Excel files"""
    try:
        df = pd.read_excel(file_path, sheet_name=0)  # Read first sheet only
        # Add metadata columns
        df['source_file'] = Path(file_path).name
        df['import_timestamp'] = pd.Timestamp.now()
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")
        return None


In [89]:
# Task 1a: EDA and Cleaning
def clean_and_analyze(df, df_name):
    """Perform EDA and cleaning on a dataframe"""
    if df is None:  # Check if the dataframe is None (not loaded properly)
        print(f"{df_name} DataFrame is not valid. Skipping analysis.")
        return df
    
    print(f"\n=== Initial Analysis for {df_name} ===")
    
    # Initial inspection
    print(f"Original shape: {df.shape}")
    print("\nColumns:", df.columns.tolist())
    print("\nData types:\n", df.dtypes)
    
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace(r'[^\w_]', '', regex=True)
    print("\nStandardized columns:", df.columns.tolist())
    
    # Handle missing values
    initial_rows = len(df)
    df.dropna(how='all', inplace=True)  # Drop completely empty rows
    rows_dropped = initial_rows - len(df)
    print(f"\nRows dropped (all NA): {rows_dropped}")
    
    # Check for duplicates
    dupes = df.duplicated().sum()
    print(f"Duplicate rows found: {dupes}")
    if dupes > 0:
        df.drop_duplicates(inplace=True)
        print(f"Dropped {dupes} duplicates")
    
    # Clean date columns if they exist
    date_cols = [col for col in df.columns if 'date' in col]
    for col in date_cols:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            print(f"Converted {col} to datetime")
        except:
            print(f"Could not convert {col} to datetime")
    
    return df


In [90]:
# Directory containing all the Excel files
excel_files_directory = r'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_one_data\\ExcelFiles'


In [91]:
file_path_pattern = f"{excel_files_directory}\\*.xlsx"


In [92]:
# Get a list of all Excel files in the directory
all_files = glob.glob(file_path_pattern)


In [93]:
# Directory where cleaned files will be saved
cleaned_files_directory = r'C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_one_data\ExcelFiles\After_cleaning'


In [94]:
# Ensure directory exists
Path(cleaned_files_directory).mkdir(parents=True, exist_ok=True)


In [95]:
# Process files and clean
cleaned_dfs = {}
for file in all_files:
    df_name = Path(file).stem  # Get the base name without extension
    print(f"Processing file: {file}")  # Debugging print to track which file is being processed
    df = read_excel_files(file)
    if df is not None:  # Ensure df is not None before cleaning
        cleaned_df = clean_and_analyze(df, df_name)
        cleaned_dfs[df_name] = cleaned_df
    else:
        print(f"Skipping file {file} due to read failure.")  # Debugging print for failed file read


Processing file: C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_one_data\\ExcelFiles\Cyber Liability Standard Loss Run_DF453172.xlsx

=== Initial Analysis for Cyber Liability Standard Loss Run_DF453172 ===
Original shape: (6, 13)

Columns: ['Cyber Liability Standard Loss Run', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'source_file', 'import_timestamp']

Data types:
 Cyber Liability Standard Loss Run            object
Unnamed: 1                                   object
Unnamed: 2                                   object
Unnamed: 3                                   object
Unnamed: 4                                   object
Unnamed: 5                                   object
Unnamed: 6                                   object
Unnamed: 7                                   object
Unnamed: 8                                   object
Unname

C:\Users\FINRISE\AppData\Local\Temp\ipykernel_10396\1044311864.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [96]:
# Save cleaned dataframes to the specified directory
for df_name, cleaned_df in cleaned_dfs.items():
    if cleaned_df is not None and not cleaned_df.empty:  # Ensure dataframe is not empty
        # Saving to the specified directory
        output_file_path = Path(cleaned_files_directory) / f'Cleaned_{df_name}.xlsx'
        cleaned_df.to_excel(output_file_path, index=False)
        print(f"Saved cleaned data for {df_name} to {output_file_path}")
    else:
        print(f"Skipping saving {df_name} as it is empty.")  # Debugging print for empty dataframes

print("\nTask completed successfully. Cleaned files have been saved.")


Saved cleaned data for Cyber Liability Standard Loss Run_DF453172 to C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_one_data\ExcelFiles\After_cleaning\Cleaned_Cyber Liability Standard Loss Run_DF453172.xlsx
Saved cleaned data for LedgerDetails to C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_one_data\ExcelFiles\After_cleaning\Cleaned_LedgerDetails.xlsx
Saved cleaned data for LedgerDetails_2 to C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_one_data\ExcelFiles\After_cleaning\Cleaned_LedgerDetails_2.xlsx
Saved cleaned data for MatterDetails_CompanyX to C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_one_data\ExcelFiles\After_cleaning\Cleaned_MatterDetails_CompanyX.xlsx
Saved cleaned data for PolicyDetails to C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\